# Challenge 7 - Fight Fire with Data
## Random Forest Model to Predict Fire Spread

The user will be using a Jupyter Notebook to run code that was developed in R. First, the user will check to see if the wind speed and brightness are correlated with the speed that the fire spreads derived from the satellite data. The input data has been prepared for you. Next, the user will run the code that creates a model (random forest) using the features they select (windspeed and brightness) as the inputs and estimates the speed of spread as the target variable (speed of spread). They will train a model, record the Root Mean Squared Error, and save the model into a deployable format also known as Predictive Model Markup Language (PMML). 


## Install and Load Packages

In [ ]:
# Install and load packages
library(projectLib)
library(httr)
library(jsonlite)
host <- "https://api.watsonwarriors.ai/workers/validate";


project <- projectLib::Project$new(,"<ProjectId>", "<ProjectToken>")

library(randomForest)
library(caret)
library(data.table)
library(devtools)

install_git("git://github.com/jpmml/r2pmml.git")
library(r2pmml)

## Get and View Data

In [ ]:
# Get data
# https://github.com/watsonwarriors2020/challenges/raw/master/Challenge_7_Merged_Data_single_fire.csv
fireData = fread('https://raw.githubusercontent.com/watsonwarriors2020/challenges/master/Challenge_7_Merged_Data_single_fire.csv'
                 , data.table=FALSE
                 , header = TRUE
                 , stringsAsFactors = FALSE)

# Attach the column names
attach(fireData)

# Show data structure
str(fireData)

## Train a Random Forest Model and Display Accuracy

In [ ]:
# Train model
set.seed(556)
modFit_rf <- randomForest(speed_mph~brightness + frp + WindSpeedMph + SurfaceTemperatureFahrenheit,
                          data = fireData,
                          # nodesize = 1,
                          ntree = 20,
                          trControl = trainControl(method = "cv", number = 10)
                          )

print(modFit_rf)

#RMSE
rmse <- round(postResample(fireData$speed_mph, predict(modFit_rf, fireData))[1],4)
answers <- list('0' = rmse)

In [ ]:
answers

## Export Predictive Model Markup Language file

In [ ]:
# Export the model to PMML
r2pmml(modFit_rf, "modFit_rf.pmml")
#Export the PMML file to project storage
project$save_data('modFit_rf.xml', "modFit_rf.pmml", overwrite=TRUE)

## Complete Challenge

In [ ]:
## Paste validation code below
POST(host,  body = list('data' = answers), encode = "json", add_headers('ww-session' = '09616428-f5fe-49df-bdf3-3a392728201a.5e334940153cad50b45ae113'))